## Introduction
### Bank Note Image Classification Model Deployment
*By Siek Ming Jun - U1820369F*

For my project, I have trained an image recognition model to identify the bank notes available in Singapore using Google's **Teachable Machine platform**. The bank notes we would be training the model with are the **2, 5, 10 and 50** dollars. Multiple photos were taken in order to increase the accuracy of the model to identify the bank notes.

Once the model has been trained, we exported the model as Tensorflow in the Keras format. With the model created, we are now able to deploy the model into **Azure Machine Learning** platform using purely python code inside this Jupyter notebook.

This notebook provides details on how to deploy your machine learning model onto the Azure Machine Learning platform in the cloud. The steps are as follows:

In [3]:
# 1. Create Workspace in Azure 
#    You will need to enter your own Azure subscription ID into the arguments below

from azureml.core import Workspace

ws = Workspace.create(name='iot-ws', # provide a name for your workspace
                      subscription_id='[subscription ID]', # provide your subscription ID
                      resource_group='iot-assignment', # provide a resource group name
                      create_resource_group=True,
                      location='southeastasia') # For example: 'westeurope' or 'eastus2' or 'westus2' or 'southeastasia'.

# write out the workspace details to a configuration file: .azureml/config.json
ws.write_config(path='.azureml')

Deploying AppInsights with name iotwsinsightsa90d1b8486d.
Deployed AppInsights with name iotwsinsightsa90d1b8486d. Took 3.81 seconds.
Deploying KeyVault with name iotwskeyvaulte2f5cc058dd.
Deploying StorageAccount with name iotwsstorage1e5117a6885e.
Deployed KeyVault with name iotwskeyvaulte2f5cc058dd. Took 19.24 seconds.
Deploying Workspace with name iot-ws.
Deployed StorageAccount with name iotwsstorage1e5117a6885e. Took 20.94 seconds.
Deployed Workspace with name iot-ws. Took 85.36 seconds.


In [69]:
# 1.1. If Workspace is already created, load Workspace from config 
# from azureml.core import Workspace

# ws = Workspace.from_config()

In [23]:
# 2. View details of workspace
ws.get_details()

{'id': '/subscriptions/4a4f907e-ad41-435e-b51d-fe4a2d4c25f7/resourceGroups/iot-assignment/providers/Microsoft.MachineLearningServices/workspaces/iot-ws',
 'name': 'iot-ws',
 'identity': {'principal_id': '42b50a6e-e8e6-4c65-98a9-e73afaf86553',
  'tenant_id': '15ce9348-be2a-462b-8fc0-e1765a9b204a',
  'type': 'SystemAssigned'},
 'location': 'southeastasia',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'sku': 'Basic',
 'workspaceid': 'f7e3d26a-bdea-4265-aeeb-a3513de624a7',
 'sdkTelemetryAppInsightsKey': '0855780c-10d5-4461-ae3c-9b15eb18c90d',
 'description': '',
 'friendlyName': 'iot-ws',
 'creationTime': '2021-02-24T05:12:11.7875389+00:00',
 'containerRegistry': '/subscriptions/4a4f907e-ad41-435e-b51d-fe4a2d4c25f7/resourceGroups/iot-assignment/providers/Microsoft.ContainerRegistry/registries/f7e3d26abdea4265aeeba3513de624a7',
 'keyVault': '/subscriptions/4a4f907e-ad41-435e-b51d-fe4a2d4c25f7/resourcegroups/iot-assignment/providers/microsoft.keyvault/vaults/iotwskeyvaulte2f5cc0

In [31]:
# 3. Register model that you trained, model should be stored under './models/bank-note-model.h5'

from azureml.core.model import Model
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory
model = Model.register(model_path = "./models/bank-note-model.h5",
                       model_name = "bank_note_ic",
                       description = "Image Classification model trained outside Azure Machine Learning",
                       workspace = ws)

Registering model bank_note_ic


### 4. Create entry script used in InferenceConfig

In [32]:
%%writefile score.py

import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np
import base64
import os
import io
import json
import glob
from keras.models import load_model

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Called when the deployed service starts
def init():
    global model

    # Get the path where the deployed model can be found.
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), './bank-note-model.h5')
    # load models
    model = load_model(model_path)

# Handle requests to the service
def run(data):
    try:
        # Pick out the text property of the JSON request.
        # This expects a request in the form of {"text": "some text to score for sentiment"}
        data = json.loads(data)
        prediction = predict(data['image_data'])
        #Return prediction
        return prediction
    except Exception as e:
        error = str(e)
        return error

def predict(image_data, include_neutral=True):
    
    note_classes = ['fifty_dollar', 'five_dollar', 'ten_dollar', 'two_dollar']
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    
    imgdata = base64.b64decode(str(image_data))
    image = Image.open(io.BytesIO(imgdata))

    #resize the image to a 224x224 with the same strategy as in TM2:
    #resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)
    
    #turn the image into a numpy array
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Load the image into the array
    data[0] = normalized_image_array

    # run the inference
    prediction = model.predict(data)
    return note_classes[prediction.argmax()]

Overwriting score.py


In [33]:
# 5. Define inference config

from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("tensorflow")
conda_dep.add_pip_package("keras")
conda_dep.add_pip_package("pillow")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)

In [8]:
# 6. Deploy to cloud

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [12]:
# 7. Retrieve existing service
# This will provide the url that your app can send a POST request to via REST 

service = Webservice.list(ws)[0]
print(service.scoring_uri)

http://302246c6-065b-4a25-8cf8-8e70627924aa.southeastasia.azurecontainer.io/score


In [34]:
# 8. Update service: If service is already running, you can updated the service using the code below instead of redeploying 
#    the entire Workspace. This should not be run during the first deployment

service.update(models=[model], inference_config=inference_config)
service.wait_for_deployment(show_output=True)
print(service.state)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
2021-02-24T07:18:59,310546300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_73af9c7ec19373116880cf0b602f9900/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_73af9c7ec19373116880cf0b602f9900/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_73af9c7ec19373116880cf0b602f9900/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_73af9c7ec19373116880cf0b602f9900/lib/libssl.so.1.0.0: no version information available (required by /usr/

In [35]:
# 9. Check the state of our service
print(service.state)
print(service.scoring_uri)

Healthy
http://302246c6-065b-4a25-8cf8-8e70627924aa.southeastasia.azurecontainer.io/score


In [38]:
# 10. The code below is used to test whether the model we have deployed in the cloud is working.
#    We send over a our test image file to our service and it will return the prediction to us.

import base64
import requests

with open("test2.jpg", "rb") as image_file:
    image_data = base64.b64encode(image_file.read())

image_data= image_data.decode('utf-8')

input_data = {"image_data": str(image_data)}
input_data = json.dumps(input_data)

# input_data = "{\"image_data\": " + image_data + "}"

headers = {'Content-Type': 'application/json'}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("prediction:", resp.text)

POST to url http://302246c6-065b-4a25-8cf8-8e70627924aa.southeastasia.azurecontainer.io/score
prediction: "ten_dollar"


In [157]:
# 11. Once we are satisfied with our deployment, we are able to delete the service if we have no need for it
#     in order to reduce cost. 
service.delete()

In [ ]:
# 12. We can delete our model as well.
model.delete()

In [112]:
# 13. We can also delete workspace and clean up resources
ws.delete()

In [2]:
# Testing of model on local machine without cloud

# import tensorflow.keras
# from PIL import Image, ImageOps
# import numpy as np
# import base64
# import os
# import json
# import io
# import base64

# data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    
# model = tensorflow.keras.models.load_model('models/bank-note-model.h5')
    
# note_classes = ['fifty_dollar', 'five_dollar', 'ten_dollar', 'two_dollar']

# with open("test2.jpg", "rb") as image_file:
#     image_data = base64.b64encode(image_file.read())
# image_data = image_data.decode('utf-8')
# image_data = str(image_data)

# imgdata = base64.b64decode(str(image_data))
# image = Image.open(io.BytesIO(imgdata))

# #resize the image to a 224x224 with the same strategy as in TM2:
# #resizing the image to be at least 224x224 and then cropping from the center
# size = (224, 224)
# image = ImageOps.fit(image, size, Image.ANTIALIAS)

# #turn the image into a numpy array
# image_array = np.asarray(image)

# # Normalize the image
# normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# # Load the image into the array
# data[0] = normalized_image_array

# # run the inference
# prediction = model.predict(data)
# print(prediction)
# json.dumps(note_classes[prediction.argmax()])

[[5.2672262e-05 1.1045220e-03 9.9774593e-01 1.0969235e-03]]


'"ten_dollar"'